# Data Cleaning - Getting Shift Data for 2018-19

This notebook will transform the play-by-play NBA data into shifts where a shift is some period of time in an NBA game where the same 10 players are on the court without substitutions. This is important so that we can get the +/- for each shift and know each player that participated in a shift.

According to the paper from Kostas, when recording shifts we need to measure both point differential and number of possessions in each shift. Then for the Bayesian regression we regress the point differential per 100 possessions from the shift onto indicators corresponding to the 10 players on the court during the shift.

In [1]:
import pandas as pd
import numpy as np

data_1819 = pd.read_csv("../data/events_2018-2019_pbp.csv")

/Users/reedpeterson/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (71,89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
data_1819.head()

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,NEUTRALDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,...,HOME_PLAYER_ID_1_PLAY_TIME,HOME_PLAYER_ID_2_PLAY_TIME,HOME_PLAYER_ID_3_PLAY_TIME,HOME_PLAYER_ID_4_PLAY_TIME,HOME_PLAYER_ID_5_PLAY_TIME,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME
0,0,0,12,2,21801052,NaN,NaN,12:00,1,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,10,4,21801052,Jump Ball Horford vs. Jokic: Tip to Millsap,NaN,12:00,1,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1,2,7,21801052,NaN,NaN,11:41,1,5.0,...,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0
3,3,0,4,8,21801052,Tatum REBOUND (Off:0 Def:1),NaN,11:39,1,4.0,...,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0
4,4,1,2,9,21801052,MISS Horford 15' Jump Shot,NaN,11:34,1,4.0,...,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0


### Useful Columns to Keep

* GAME_ID
* PCTIMESTRING (time left in quarter)
* PERIOD
* SCOREMARGIN
* POSSESSION_ID - note - this variable is useless. In 2018-19 datatset this variable is literally all nan values.
* HOME_PLAYER_ID_{1:5}
* AWAY_PLAYER_ID_{1:5}
* SUB_ENTERED_PLAYER_ID - this will tell us when a shift ends (shift ends when a sub or multiple subs come in, new shift starts)
* HOME_TEAM
* AWAY_TEAM

To identify number of possessions, we will use the formula from this website: https://www.nbastuffer.com/analytics101/possession/

For this formula we need: 

* Total field goal attempts in the shift
* Total turnovers in the shift
* Total free throw attempts in the shift
* Total offensive rebounds in the shift

Note - we can eventually match player IDs to player names using the "playerlist.csv" file. For now though we can work with arbitrary player IDs. 

In [3]:
data_1819.loc[~pd.isna(data_1819.REBOUND_PLAYER_ID)][['REBOUND_TEAM', 'REBOUND_PLAYER_ID']]

# data_1819['REBOUND_TEAM'] 31, 58, 81, 96, 105

# data_1819.iloc[96].REBOUND_PLAYER_ID

# SHOT_PLAYER_ID when not null, we have a FG attempt. Also must record if this player is for the home or away team
# TURNOVER_PLAYER_ID when not null, we have a turnover
# FREE_THROW_PLAYER_ID when not null, we have a free throw
# REBOUND_PLAYER_ID when not null, we have a rebound - then we must check if this player id is in the list of players on the same team as the shooting player ID
# i.e. if the SHOT_PLAYER_ID was an away player, we must check that REBOUND_PLAYER_ID is also an away player in order to say that this was an offensive rebound

# so I'll need to keep track of the team of the most recent shooter. Then if we see a rebound_player_id we check if that id is from the same team as the most recent shooter

,REBOUND_TEAM,REBOUND_PLAYER_ID
3,NaN,1.628369e+06
5,NaN,2.011430e+05
7,NaN,2.039990e+05
11,NaN,2.026810e+05
14,NaN,2.011430e+05
...,...,...
582443,NaN,2.015770e+05
582445,True,1.610613e+09
582456,NaN,2.019540e+05
582463,NaN,1.627782e+06


## The cell below gets a subset of columns from the dataset with only the variables that we need for creating shifts

In [4]:
subdata = data_1819[['GAME_ID', 'HOME_TEAM', 'AWAY_TEAM', 'PCTIMESTRING', 'PERIOD', 'SCOREMARGIN', 'SUB_ENTERED_PLAYER_ID', 'SHOT_PLAYER_ID', 'TURNOVER_PLAYER_ID', 'FREE_THROW_PLAYER_ID', 'REBOUND_PLAYER_ID', 'HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']]

# subdata.loc[~pd.isna(subdata.SCOREMARGIN)] # need to replace "TIE" with 0 
subdata

,GAME_ID,HOME_TEAM,AWAY_TEAM,PCTIMESTRING,PERIOD,SCOREMARGIN,SUB_ENTERED_PLAYER_ID,SHOT_PLAYER_ID,TURNOVER_PLAYER_ID,FREE_THROW_PLAYER_ID,...,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5
0,21801052,Celtics,Nuggets,12:00,1,NaN,NaN,NaN,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
1,21801052,Celtics,Nuggets,12:00,1,NaN,NaN,NaN,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
2,21801052,Celtics,Nuggets,11:41,1,NaN,NaN,200794.0,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
3,21801052,Celtics,Nuggets,11:39,1,NaN,NaN,NaN,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
4,21801052,Celtics,Nuggets,11:34,1,NaN,NaN,201143.0,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582463,21800959,Bulls,Pacers,0:20,4,NaN,NaN,NaN,NaN,NaN,...,1626167,202083,202711,201954,201152,1627885,1627789,1628469,1627782,1627853
582464,21800959,Bulls,Pacers,0:20,4,NaN,NaN,NaN,NaN,NaN,...,1626167,202083,202711,201954,201152,1627885,1627789,1628469,1627782,1627853
582465,21800959,Bulls,Pacers,0:14,4,NaN,NaN,1627789.0,NaN,NaN,...,1626167,202083,202711,201954,201152,1627885,1627789,1628469,1627782,1627853
582466,21800959,Bulls,Pacers,0:11,4,NaN,NaN,NaN,NaN,NaN,...,1626167,202083,202711,201954,201152,1627885,1627789,1628469,1627782,1627853


## The cell below fixes an issue with multiple data types in the SCOREMARGIN column

In [5]:
# subdata.replace("TIE", 0, inplace = True)
# subdata.loc[~pd.isna(subdata.SCOREMARGIN)]

new_scoremargin = subdata.SCOREMARGIN.replace("TIE", 0)

# subdata.SCOREMARGIN = new_scoremargin # This works but gives some warning message
# subdata.assign(SCOREMARGIN = new_scoremargin, inplace = True)
# subdata['SCOREMARGIN'] = new_scoremargin
subdata.drop("SCOREMARGIN", axis = 1, inplace = True)
subdata.insert(3, 'SCOREMARGIN', new_scoremargin, True)

/Users/reedpeterson/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
# check that it worked
subdata.loc[~pd.isna(subdata.SCOREMARGIN)] # need to replace "TIE" with 0 
# subdata

,GAME_ID,HOME_TEAM,AWAY_TEAM,SCOREMARGIN,PCTIMESTRING,PERIOD,SUB_ENTERED_PLAYER_ID,SHOT_PLAYER_ID,TURNOVER_PLAYER_ID,FREE_THROW_PLAYER_ID,...,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5
9,21801052,Celtics,Nuggets,2,11:16,1,NaN,202681.0,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
12,21801052,Celtics,Nuggets,4,10:51,1,NaN,1628369.0,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
20,21801052,Celtics,Nuggets,2,10:05,1,NaN,203999.0,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
29,21801052,Celtics,Nuggets,0,9:06,1,NaN,203999.0,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
33,21801052,Celtics,Nuggets,-2,8:17,1,NaN,203115.0,NaN,NaN,...,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582450,21800959,Bulls,Pacers,6,0:45,4,NaN,NaN,NaN,1628374.0,...,1626167,202083,202711,201954,201152,203897,203490,1628374,1627782,1627853
582453,21800959,Bulls,Pacers,7,0:34,4,NaN,NaN,NaN,201954.0,...,1626167,202083,202711,201954,201152,203897,203490,1628374,1627782,1627853
582454,21800959,Bulls,Pacers,8,0:34,4,NaN,NaN,NaN,201954.0,...,1626167,202083,202711,201954,201152,203897,203490,1628374,1627782,1627853
582458,21800959,Bulls,Pacers,9,0:20,4,NaN,NaN,NaN,201954.0,...,1626167,202083,202711,201954,201152,203897,203490,1628374,1627782,1627853


In [7]:
data_1819.loc[~pd.isna(data_1819.SUB_ENTERED_PLAYER_ID)][['SUB_ENTERED_PLAYER_ID', 'GAME_ID', 'PCTIMESTRING', 'PERIOD', 'SCOREMARGIN', 'SHOT_PLAYER_ID', 'TURNOVER_PLAYER_ID', 'FREE_THROW_PLAYER_ID', 'REBOUND_PLAYER_ID', 'HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']]

data_1819[['SUB_ENTERED_PLAYER_ID', 'GAME_ID', 'PCTIMESTRING', 'PERIOD', 'SCOREMARGIN', 'SHOT_PLAYER_ID', 'TURNOVER_PLAYER_ID', 'FREE_THROW_PLAYER_ID', 'REBOUND_PLAYER_ID', 'HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']].head(99)


,SUB_ENTERED_PLAYER_ID,GAME_ID,PCTIMESTRING,PERIOD,SCOREMARGIN,SHOT_PLAYER_ID,TURNOVER_PLAYER_ID,FREE_THROW_PLAYER_ID,REBOUND_PLAYER_ID,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5
0,NaN,21801052,12:00,1,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
1,NaN,21801052,12:00,1,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
2,NaN,21801052,11:41,1,NaN,200794.0,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
3,NaN,21801052,11:39,1,NaN,NaN,NaN,NaN,1.628369e+06,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
4,NaN,21801052,11:34,1,NaN,201143.0,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,NaN,21801052,1:09,1,-3,1627759.0,NaN,NaN,NaN,202681,1626179,201143,203382,1627759,1628420,1627736,202738,203486,200794
95,NaN,21801052,0:54,1,NaN,202738.0,NaN,NaN,NaN,202681,1626179,201143,203382,1627759,1628420,1627736,202738,203486,200794
96,NaN,21801052,0:52,1,NaN,NaN,NaN,NaN,1.610613e+09,202681,1626179,201143,203382,1627759,1628420,1627736,202738,203486,200794
97,202694.0,21801052,0:52,1,NaN,NaN,NaN,NaN,NaN,202681,1626179,202694,203382,1627759,1628420,1627736,202738,203486,200794


# The following two cells are the main cells for creating shifts

In [8]:
def calculate_possessions(fga, to, fta, oreb):
    """
    This function takes in field goal attemps (fga), turnovers (to), free throw attempts (fta), and offensive rebounds 
    (oreb) and outputs the number of possessions from this run of play.
    """
    return 0.96 * (fga + to + 0.44 * fta - oreb)

In [9]:
# Initialize some variables that will keep track of various statistics to be used for computing possessions and point differential for shifts
home_players = set()
away_players = set()
# initialize home and road team variables
home_team = None
away_team = None
new_shift = True
game_id = None
initial_margin = 0 # this will keep track of the score margin at the start of each shift. Note: score margin = home - road scores
final_margin = 0 # this will keep track of the score margin at the end of each shift.
fg_attempts = 0 # field goal attempts
turnovers = 0
free_throws = 0
oreb = 0 # offensive rebounds
last_shooter = 0 # this keeps track of the most recent field goal shooter so we can check if the subsequent rebound is offensive or defensive

results = [] # initialize results list. We will append lists to this, so it will end up as a 2d list which we can then cast to a dataframe.

for i in range(len(subdata)):
    
    if i % 10000 == 0: print(i)
        
    if new_shift:
        # clear old players from sets and add new players for this new shift
        home_players.clear()
        away_players.clear()
        # add home players
        for j in range(1, 6):
            player = subdata['HOME_PLAYER_ID_' + str(j)].iloc[i]
            home_players.add(player)
        # add away players
        for j in range(1, 6):
            player = subdata['AWAY_PLAYER_ID_' + str(j)].iloc[i]
            away_players.add(player)
            
        # reset home and away teams
        home_team = subdata.iloc[i]['HOME_TEAM']
        away_team = subdata.iloc[i]['AWAY_TEAM']
            
        new_shift = False
        
        # reset these stats
        fg_attempts = 0 
        turnovers = 0
        free_throws = 0
        oreb = 0 
        initial_margin = final_margin # the new initial margin gets reset to the old final margin
        
        # NOTE - DO NOT RESET last_shooter UNLESS IT'S A NEW GAME
        cur_game_id = subdata.iloc[i]['GAME_ID']
        if cur_game_id != game_id: # then we have a new game
            game_id = cur_game_id # reset game_id
            last_shooter = 0
            initial_margin = 0
        
    # if a sub enters then we start a new shift and record the old shift
    
    cur_game_id = subdata.iloc[i]['GAME_ID']
    
    # two ways for a shift to end: substitution or new game
    if (not pd.isna(subdata.iloc[i]['SUB_ENTERED_PLAYER_ID'])) or cur_game_id != game_id:
        new_shift = True
 
        # record total point differential
        point_diff = int(final_margin) - int(initial_margin)
        
        # record total number of possessions
        num_possessions = calculate_possessions(fg_attempts, turnovers, free_throws, oreb)
        
        # if we somehow end up in a strange situation with zero possessions, then just continue and don't bother saving this data
        if num_possessions == 0:
            continue
            
        # add this data to some results dataframe
        shift = [point_diff, num_possessions, home_team, away_team]
        for player in home_players: # add home players to shift
            shift.append(player)
        for player in away_players: # add away players to shift
            shift.append(player)
        
        results.append(shift) # add this shift to our results list
    
    else:
        # first check to update the final_margin - this will be kept on a rolling basis (each time we see a new score margin this is the current final margin)
        if not pd.isna(subdata.iloc[i]['SCOREMARGIN']):
            final_margin = subdata.iloc[i]['SCOREMARGIN']
            
        # start aggregating point fg attempts, ft attempts, o-rebounds, and turnovers
        if not pd.isna(subdata.iloc[i]['SHOT_PLAYER_ID']):
            fg_attempts += 1
            last_shooter = subdata.iloc[i]['SHOT_PLAYER_ID'] # update most recent shooter
        if not pd.isna(subdata.iloc[i]['TURNOVER_PLAYER_ID']):
            turnovers += 1
        if not pd.isna(subdata.iloc[i]['FREE_THROW_PLAYER_ID']):
            free_throws += 1
            last_shooter = subdata.iloc[i]['FREE_THROW_PLAYER_ID'] # update most recent shooter for free throws as well
        if not pd.isna(subdata.iloc[i]['REBOUND_PLAYER_ID']):
            # now we need to check if the rebounder is on the same team as the most recent shooter
            rebounder = subdata.iloc[i]['REBOUND_PLAYER_ID']
            # if last shooter and rebounder are on the same team - increment offensive rebounds
            if ((last_shooter in home_players) and (rebounder in home_players)) or ((last_shooter in away_players) and (rebounder in away_players)):
                oreb += 1
                
        
# Make sure we also append the last shift when we reach the end of the dataset

# record total point differential
point_diff = int(final_margin) - int(initial_margin)

# record total number of possessions
num_possessions = calculate_possessions(fg_attempts, turnovers, free_throws, oreb)

# if we somehow end up in a strange situation with zero possessions, then just continue and don't bother saving this data
if num_possessions == 0:
    go = False
else:
    go = True

if go:
    # add this data to some results dataframe
    shift = [point_diff, num_possessions]
    for player in home_players: # add home players to shift
        shift.append(player)
    for player in away_players: # add away players to shift
        shift.append(player)

    results.append(shift) # add this shift to our results list
            




0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000


## NOTE - only run the following cell if the previous cell above gave an error after exiting the main loop

In [126]:
# NOTE - only run this cell if the above cell gave an error message after exiting the main loop

point_diff = int(final_margin) - int(initial_margin)

# record total number of possessions
num_possessions = calculate_possessions(fg_attempts, turnovers, free_throws, oreb)

# if we somehow end up in a strange situation with zero possessions, then just continue and don't bother saving this data
if num_possessions == 0:
    go = False
else:
    go = True

if go:
    # add this data to some results dataframe
    shift = [point_diff, num_possessions]
    for player in home_players: # add home players to shift
        shift.append(player)
    for player in away_players: # add away players to shift
        shift.append(player)

    results.append(shift) # add this shift to our results list

In [127]:
np.shape(results)

(33891, 12)

In [11]:
shifts_df = pd.DataFrame(results)
shifts_df.columns = ['point_differential', 'num_possessions', 'home_team', 'away_team', 'home_player_1', 'home_player_2', 'home_player_3', 'home_player_4', 'home_player_5', 'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4', 'away_player_5']
shifts_df

,point_differential,num_possessions,home_team,away_team,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5
0,-7,19.2000,Celtics,Nuggets,202694,1628369,201143,202681,203935,1627750,203914,203115,200794.0,203999.0
1,3,7.6800,Celtics,Nuggets,1628369,203382,201143,202681,203935,1627750,203914,203115,200794.0,203999.0
2,-1,1.3824,Celtics,Nuggets,1627759,203382,201143,202681,203935,1627750,203914,203115,200794.0,203999.0
3,-1,2.7648,Celtics,Nuggets,1627759,203382,201143,202681,203935,1628420,203914,203115,200794.0,203999.0
4,2,5.2224,Celtics,Nuggets,1627759,203382,201143,202681,203935,1628420,203115,1627736,203486.0,203999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33886,7,18.0096,Bulls,Pacers,201152,201954,202083,1626167,202711,203490,201577,1628374,203897.0,1627739.0
33887,2,3.3024,Bulls,Pacers,201152,201954,202083,1626167,202711,203490,1627782,1628374,203897.0,1627739.0
33888,-1,1.3824,Bulls,Pacers,201152,201954,202083,1626167,202711,203490,201577,1628374,203897.0,1627739.0
33889,2,2.6496,Bulls,Pacers,201152,201954,202083,1626167,202711,203490,1627782,1627853,1628374.0,203897.0


### Run the next cell to get a csv file of shift data

In [12]:
shifts_df.to_csv(r'../data/shifts_data_2018_19.csv')